# Esercizio 2

Accedere a https://www.kaggle.com/rounakbanik/the-movies-dataset#movies_metadata.csv e scaricare il dataset in formato csv `movies_metadata.csv.zip`. Tale dataset contiene, in *record* di 24 campi, i film realizzati fino al 2017.

Considerando solo i primi 1000 record, si richiede di:
- elencare i 10 paesi che hanno prodotto più film, ciascuno con il numero di film prodotti (ordinati per numero di film prodotti decrescente)
- per ognuno dei generi cinematografici presenti:
    - la classifica degli n film più popolari ordinati per popolarità decrescente, specificando il titolo originale e la tagline per ciascuno di essi
    - l'elenco delle lingue originali che sono coinvolte in ciascuna delle classifiche del punto precedente

Parametri di input:

- nome del dataset in input
- parametro n

***

I campi del file csv che occorrono per risolvere l'esercizio sono:

- `genres` (stringa)

        [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]

- `original_title` (stringa)
- `popularity` (decimale)
- `id` (ID)
- `tagline` (stringa)
- `original_language` (stringa)
- `production_countries` (stringa)

        [{'iso_3166_1': 'DE', 'name': 'Germany'}, {'iso_3166_1': 'US', 'name': 'United States of America'}]

***

Requisiti generali:

- le stringhe relative ai campi `genres` e `production_countries` *letteralmente* specificano una lista di dizionari in cui è presente la chiave `name` a cui viene associato il valore di un genere cinematografico nel caso di `genres` e un paese nel caso di `production_countries`. Si richiede quindi la definizione di una funzione `get_items()` che prenda in input uno qualsiasi tra questi due campi ed estragga la lista dei generi (ad esempio `['Animation', 'Comedy', 'Family']`) nel caso di `genres` e la lista dei paesi di produzione (ad esempio `['Germany', 'United States of America']`) nel caso di `production_countries`.
- si richiede produrre le seguenti variabili:
    - `country_rank_list`: lista di 10 tuple *(paese, numero di film prodotti)* contenente i primi 10 paesi che hanno prodotto più film (ordinati per numero di film prodotti decrescente)
- `pop_rank_dict`: dizionario delle classifiche per genere dei primi n film ordinati per popolarità decrescente
    - *chiave*: genere di un film
    - *valore*: lista di n liste *[titolo originale, tagline]* contenente i primi n film ordinati per popolarità decrescente
- `language_set_dict`: dizionario degli insiemi delle lingue coinvolte in ciascuna delle classifiche presenti in `pop_rank_dict`
    - *chiave*: genere di un film
    - *valore*: insieme delle lingue coinvolte


**NOTA BENE**: alcuni record del dataset csv hanno un valore del campo `popularity` che non è un decimale oppure non hanno proprio un valore. Non conviene perciò considerarle.

***

## Soluzione

### Parametri in input

In [2]:
input_file_name = './movies_metadata.csv.zip'
n = 15

Importazione dei moduli `pandas`, `ast` e `numpy`.

In [3]:
import pandas as pd
import ast
import numpy as np

### Definizione della funzione `get_items()`

La funzione `get_items()` estrae la lista dei generi/paesi dai campi `genres`/`production_countries` del dataset csv

In [4]:
def get_items(arg_string):
    return [dictionary['name'] for dictionary in ast.literal_eval(arg_string)]

### Lettura del dataset csv in un *data frame* `movies_df`

Leggere il dataset csv ed estrarre solo le prime 1000 righe.

In [7]:
movies_df = pd.read_csv(input_file_name)[:1000]

In [41]:
#movies_df.head()

### Costruzione delle strutture dati di base

Costruire le seguenti strutture di base:
- `info_dict`: dizionario delle informazioni sui film
    - *chiave*: ID dei film (campo `id` del dataset csv)
    - *valore*: tupla *(titolo originale, tagline, lingua originale)*
- `country_list`: lista dei paesi che hanno prodotto un film (ciascun paese deve essere presente nella lista esattamente per il numero di volte in cui ha prodotto un film)
- `pop_dict`: dizionario delle popolarità
    - *chiave*: genere cinematografico
    - *valore*: lista dei film associati al genere (ognuno dei film è presente come lista annidata *[popolarità, ID]*

Inizializzare le strutture di base vuote.

In [9]:
info_dict = {}
country_list = []
pop_dict = {}

Attraversare i *record* di `movies_df` per riempire `info_dict` e `country_list`, e costruire `pop_dict` con valori tutti uguali a liste vuote `[]`.

**NOTA BENE**: effettuare il controllo sul valore del campo `popularity` che deve essere un numero (altrimenti non considerare il record).

In [21]:
for (index, row) in movies_df.iterrows():
    if(isinstance(row['popularity'], float) and np.isnan(row['popularity']) == False):        
        info_dict[row['id']] = (row['original_title'], row['tagline'], row['original_language'])
        country_list.extend(get_items(row['production_countries']))
        [pop_dict.update([(gen, [])]) for gen in get_items(row['genres'])]

In [42]:
#info_dict

In [43]:
#country_list

In [44]:
#pop_dict

Attraversare di nuovo i *record* di `movies_df` per riempire i valori di `pop_dict`.

In [25]:
for (index, row) in movies_df.iterrows():
    if(isinstance(row['popularity'], float) and np.isnan(row['popularity']) == False):        
        [pop_dict[gen].append([row['popularity'], row['id']]) for gen in get_items(row['genres'])]

In [45]:
#pop_dict

### Estrazione dei 10 paesi che hanno prodotto più film

Costruire la lista `country_rank_list` dei primi 10 paesi che hanno prodotto più film, ordinati per numero di film decrescente; ogni paese deve essere presente come tupla *(paese, numero di film prodotti)*.

In [27]:
from collections import Counter

country_rank_counter = Counter(country_list)
country_rank_list = country_rank_counter.most_common()[:10]

In [46]:
#country_rank_list

### Estrazione, per ogni genere cinematografico, degli n film più popolari ordinati per popolarità descrescente, ed estrazione delle lingue coinvolte in ciascuna classifica di popolarità

Ottenere da `pop_dict` il dizionario `pop_rank_dict_temp` con la stessa struttura, ma con la differenza che il *valore* relativo a un dato genere è la lista degli n film più popolari (ordinati per popolarità decrescente).

**NOTA BENE**: ottenere prima `key_value_tuples`, lista di tuple *(chiave, valore)* di `pop_rank_dict_temp`, e in seguito costruire il dizionario con la funzione `dict()`.

In [39]:
key_value_tuples = [(gen, sorted(pop_dict[gen])[::-1][:n]) for gen in pop_dict]
pop_rank_dict_temp = dict(key_value_tuples)

In [47]:
#pop_rank_dict_temp

Ottenere da `pop_rank_dict_temp` il dizionario `pop_rank_dict`
- *chiave*: genere cinematografico
- *valore*: lista degli n film più popolari, dove ogni film è presente come lista annidata *[titolo originale, tagline]*

**NOTA BENE**: ottenere prima `key_value_tuples`, lista di tuple *(chiave, valore)* di `pop_rank_dict`, e in seguito costruire il dizionario con la funzione `dict()`.

In [35]:
key_value_tuples = [(gen, [[info_dict[films[1]][0], info_dict[films[1]][1]] for films in pop_rank_dict_temp[gen]]) for gen in pop_rank_dict_temp]
pop_rank_dict = dict(key_value_tuples)

In [48]:
#pop_rank_dict

Ottenere da `pop_rank_dict_temp` il dizionario `language_set_dict` degli insiemi delle lingue coinvolte in ciascuna delle classifiche per genere degli n film ordinati per popolarità descrescente
- *chiave*: genere cinematografico
- *valore*: insieme delle lingue originali coinvolte (oggetto di tipo `set`)

**NOTA BENE**: ottenere prima `key_value_tuples`, lista di tuple *(chiave, valore)* di `language_set_dict`, e in seguito costruire il dizionario con la funzione `dict()`.

In [37]:
key_value_tuples = [(gen, set([info_dict[films[1]][2] for films in pop_rank_dict_temp[gen]])) for gen in pop_rank_dict_temp]
language_set_dict = dict(key_value_tuples)

In [49]:
#language_set_dict